In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    print(filters)

64
64
64
128
128
128
128
256
256
256
256
256
256
512
512
512


# Create the residual unit

In [3]:
# create the default conv2d layer
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,kernel_size=(3,3),strides=1,
                        padding='SAME',use_bias=False)

In [4]:
# create residual layers

class ResidualUnit(keras.layers.Layer):
    #constructor
    def __init__(self,filters,strides=1,activation='relu',**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        #main layer
        self.main_layer = [
            DefaultConv2D(filters,strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters,strides=1),
            keras.layers.BatchNormalization()
        ]
        #skip layers
        self.skip_layers=[]# jika tidak => input -> mendekati identity function (h(x))
        if strides > 1 : # unsize ussualy get on transisition filters layers with halve strides 
            self.skip_layers = [
                DefaultConv2D(filters,kernel_size=(1,1),strides=strides),
                keras.layers.BatchNormalization()
            ]
    
    def call(self,inputs):
        # main pathway layer
        Z = inputs
        for layer in self.main_layer:
            Z = layer(Z)
        # skip layer
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)
        

In [5]:
# add the model 
model = keras.models.Sequential()
#regular layer
model.add(DefaultConv2D(64,kernel_size=(7,7),strides=2,
                        padding='same',input_shape=[224,224,3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(pool_size=(3,3),strides=2,padding='SAME'))
# skip layer
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters,strides=strides))
    prev_filters = filters

model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10,activation='softmax'))


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      9408      
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 64)       0         
 )                                                               
                                                                 
 residual_unit (ResidualUnit  (None, 56, 56, 64)       74240     
 )                                                               
                                                        